In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_9697.jpg
/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_10087.jpg
/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_9618.jpg
/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_9614.jpg
/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_9862.jpg
/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_9838.jpg
/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_9851.jpg
/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_9691.jpg
/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_10011.jpg
/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_9785.jpg
/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test/benign/melanoma_9882.jpg
/kaggle/input/skin-

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTModel
import timm

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
# Dataset Paths
train_data_path = '/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/train'
test_data_path = '/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset/test'

# Load Data
train_dataset = datasets.ImageFolder(root=train_data_path, transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(root=test_data_path, transform=data_transforms['test'])

# Dataloaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, GRU, Dense, 
                                     Dropout, Flatten, LayerNormalization, MultiHeadAttention, Reshape, BatchNormalization)

# ✅ Hybrid Model (GRU + CNN)
inputs = Input(shape=(224, 224, 3))

# ✅ CNN Feature Extractor
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)  # Flatten for GRU

# ✅ Reshape for GRU Processing
x = Reshape((1, -1))(x)  # Reshape to (batch_size, timesteps=1, features)

# ✅ GRU Layer
x = GRU(128, return_sequences=True)(x)

# ✅ Multi-Head Attention
attn_output = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
x = LayerNormalization()(attn_output)

# ✅ Fully Connected Layers
x = Flatten()(x)  # Flatten for Dense Layer
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)  # Binary classification

# ✅ Create the Model
gru_cnn_hybrid_model = Model(inputs, outputs)

# ✅ Compile the Model
gru_cnn_hybrid_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ✅ Model Summary
gru_cnn_hybrid_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 224, 224, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 112, 112, 32)   │              0 │ conv2d[0][0]           │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 112, 112, 64)   │         18,496 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 56, 56, 64)     │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 56, 56, 128)    │         73,856 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 28, 28, 128)    │              0 │ conv2d_2[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 100352)         │              0 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 1, 100352)      │              0 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru (GRU)                 │ (None, 1, 128)         │     38,585,088 │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 1, 128)         │         66,048 │ gru[0][0], gru[0][0]   │
│ (MultiHeadAttention)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 1, 128)         │            256 │ multi_head_attention[… │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 128)            │              0 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         16,512 │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │            129 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 38,761,281 (147.86 MB)

 Trainable params: 38,761,281 (147.86 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# ✅ Training & Evaluation Function
def train_and_evaluate(model, train_gen, test_gen, epochs=10):
    # Train the model
    history = model.fit(train_gen, epochs=epochs, validation_data=test_gen)

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(test_gen)
    print(f'Test Accuracy: {test_accuracy:.4f}')

    # Predictions
    y_true = test_gen.classes  # True labels
    y_pred_probs = model.predict(test_gen)  # Probability predictions
    y_pred_binary = (y_pred_probs > 0.5).astype(int).flatten()  # Convert to binary predictions

    # ✅ Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_binary)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Melanoma', 'Melanoma'], yticklabels=['No Melanoma', 'Melanoma'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    # ✅ Classification Report (Precision, Recall, F1-Score)
    report = classification_report(y_true, y_pred_binary, target_names=['No Melanoma', 'Melanoma'])
    print(report)

    # ✅ ROC Curve and AUC Score
    fpr, tpr, _ = roc_curve(y_true, y_pred_probs)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')

    # Plot ROC Curve
    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})', color='blue')
    plt.plot([0, 1], [0, 1], 'r--')  # Diagonal reference line
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 80-20 train-validation split

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset',  # Replace with your dataset path
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

test_generator = train_datagen.flow_from_directory(
    '/kaggle/input/skin-cancer-melanoma/melanoma_cancer_dataset',  # Replace with your dataset path
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# ✅ Train & Evaluate Optimized Model
train_and_evaluate(gru_cnn_hybrid_model, train_generator, test_generator, epochs=10)


Found 8484 images belonging to 2 classes.
Found 2121 images belonging to 2 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


266/266 ━━━━━━━━━━━━━━━━━━━━ 87s 290ms/step - accuracy: 0.8829 - loss: 0.3552 - val_accuracy: 0.9057 - val_loss: 0.3129
Epoch 2/10
266/266 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - accuracy: 0.9071 - loss: 0.3275 - val_accuracy: 0.9057 - val_loss: 0.3124
Epoch 3/10
266/266 ━━━━━━━━━━━━━━━━━━━━ 21s 76ms/step - accuracy: 0.9078 - loss: 0.3236 - val_accuracy: 0.9057 - val_loss: 0.3127
Epoch 4/10
266/266 ━━━━━━━━━━━━━━━━━━━━ 20s 74ms/step - accuracy: 0.9036 - loss: 0.3340 - val_accuracy: 0.9057 - val_loss: 0.3124
Epoch 5/10
266/266 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9107 - loss: 0.3190 - val_accuracy: 0.9057 - val_loss: 0.3138
Epoch 6/10
266/266 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9056 - loss: 0.3258 - val_accuracy: 0.9057 - val_loss: 0.3129
Epoch 7/10
266/266 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - accuracy: 0.9050 - loss: 0.3276 - val_accuracy: 0.9057 - val_loss: 0.3129
Epoch 8/10
266/266 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9040 - loss: 0.3321 - val_accura

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (32, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step


NameError: name 'confusion_matrix' is not defined